# Importing the dependencies

In [3]:
import pandas as pd
import chromadb
import sqlite3

import google.generativeai as genai

import os
from dotenv import load_dotenv


load_dotenv()
GOOGLE_API_KEY_MOVIE_RECOMMENDER = os.getenv("GOOGLE_API_KEY_MOVIE_RECOMMENDER")
genai.configure(api_key=GOOGLE_API_KEY_MOVIE_RECOMMENDER)

# Chroma DB and Key Based Database Utilities

In [4]:
keyword_data_path = "../Chroma_Database"
client_key = chromadb.PersistentClient(path = keyword_data_path)
movie_collection = client_key.get_collection("Movies")

In [27]:
# Find similar movies based on the movie title from the Chroma database
def find_similar_movies(movie_title, data, movie_collection, top_k=6):
    try:
        movie_row = data[data['title'] == movie_title]
        
        if movie_row.empty:
            return f"Movie title '{movie_title}' not found in the database."
        

        query_embedding = eval(movie_row.iloc[0]['embeddings'])
        
        results = movie_collection.query(
            query_embeddings=[query_embedding],
            n_results=top_k
        )
        
        final_movies = []
        for movie in results['metadatas'][0]:
            final_movies.append(movie)
            
        return final_movies[1:]
        

    except Exception as e:
        print(f"Error occurred during request: {e}")
        return []

# SQL Database Inferencing Utilities

In [6]:
database = "../SQL_Database/Movies.db"
database_key_based = pd.read_sql_query("SELECT m.* FROM Movies_Key_Based AS m", sqlite3.connect(database))
database_query_based = pd.read_sql_query("SELECT m.* FROM Movies_Database AS m", sqlite3.connect(database))

In [31]:
def process_output(output):
    try:
        if((type(output))==str):
            print("Please enter a valid SQL command")
        else:
            for movie in output:
                print(movie[0])
    except Exception as e:
        print(f"Error occurred during request: {e}")
        


def process_sql_query(sql_command,db):
    try: 
        if(type(sql_command)==str):   
            if(sql_command=="SELECT m.* FROM Movies_Database AS m"):
                return process_output(database_query_based.values)
            
            con = sqlite3.connect(db)
            cur = con.cursor()
            
            cur.execute(sql_command)
            
            rows = cur.fetchall()
            list_of_lists = [[row[i] for i in range(len(row))] for row in rows]

            con.close()
            return process_output(list_of_lists)
        elif(type(sql_command)==list):
            for movie in sql_command:
                print(movie["title"]) # Trial
        else:
            return "Please enter a valid SQL command"
    except Exception as e:
        print(f"Error occurred during request: {e}")
        return ""

# Prompt Generation and Knowledge Base Utilities

In [8]:
prompts = [
    """
    You are an expert at converting English text to SQL code. Here, the SQL Database you are getting is called Movies_Database and 
    has the following columns : \n\n
    
        1. id
        2. IMDB_ID
        3. title
        4. release_year
        5. genres
        6. vote_average
        7. cast
        8. Director
        9. keywords
        10. reviews
        11. review_sentiment
        12. review_summary
        13. poster_path
        14. backdrop_path
        
        
    \n\nNow, the above columns which I have mentioned are the columns of the table Movies_Database and are exactly the same as they are i.e. are case sensitive.
    So, make sure you do not make any mistakes in the column names.\n\n
    
    \n\nNow, if the text query you get talks about recommending movies based on release_year, vote_average and/or Director you need to convert 
    it into a sql command as shown in the example below. Since these are strings/numeric values, you have to convert it as it is.\n\n
    
        
    \n\nTake this for an example. If the text says, "Recommend me some movies released in 2019", the SQL command for that 
    would be something like this:
    
        SELECT m.title from Movies_Database AS m where m.release_year=2019 LIMIT 0,10;
        
    \n\nTake this for an example. If the text says, "Recommend me all movies released in or after 2018", the SQL command for that 
    would be something like this:
    
        SELECT m.title from Movies_Database AS m where m.release_year>=2018;
        
    \n\nTake this for an example. If the text says, "Recommend me a movie released before 2019", the SQL command for that 
    would be something like this:
    
        SELECT m.title from Movies_Database AS m where m.release_year < 2019 LIMIT 0,1;
        
    \n\nTake this for an example. If the text says, "Recommend me 5 movies with voter average above 8", the SQL command for that 
    would be something like this:
    
        SELECT m.title from Movies_Database AS m where m.vote_average > 8.00 LIMIT 0,5;
        
    \n\nTake this for an example. If the text says, "Recommend me some movies with highly rated movies (greather than equal to 9)", the SQL command for that 
    would be something like this:
    
        SELECT m.title from Movies_Database AS m where m.vote_average >= 9.00 LIMIT 0,10;
        
    \n\nTake this for an example. If the text says, "Recommend me movies directed by James Gunn", the SQL command for that 
    would be something like this:
    
        SELECT m.title from Movies_Database AS m where m.Director == 'James Gunn';
        
    \n\nTake this for an example. If the text says, "Recommend me the latest movies by Christopher Nolan", the SQL command for that 
    would be something like this:
    
        SELECT m.title from Movies_Database AS m where m.Director == 'Christopher Nolan' ORDER BY m.release_year DESC;
        
    
    \n\n In the case of text queries which do not have any of the above mentioned columns, and ask 
    the query based on title, genres, cast and/or keywords, you just need to return all the rows as they are. These columns ARE NOT your expertise and you need to return the
    thing which the query is asking for. You have to parse the user entered query and understand where the query is pointing to. If the query is pointing to the columns of
    genres, cast and keywords, you need to just return which thing they are asking for. You do not need to convert them into SQL commands.
    
    \n\nTake this for an example. If the text says, "Recommend me some movies like Guardians of the Galaxy Vol. 2", the string output command for that is :
    
        TITLE = ['Guardians of the Galaxy Vol. 2']
        
    \n\nTake this for an example. If the text says, "Recommend me some movies similar to Avengers: Infinity War", the string output command for that is :
    
        TITLE = ['Avengers: Infinity War']
    
    \n\n Whatever genre is asked for in the user's intent, you need to map it to the below given list of genres in the database and return the movies based on that. Extract that which genre is the user's intent to search, and use the closest match from the below given list of genres to return the movies.
    
    genres_data = ['family','romance','action','war','comedy','sci-fi','mystery','animation','documentary','western','history','adventure','movie','fantasy','music','horror','crime','drama','tv','thriller']
           
    \n\nTake this for an example. If the text says, "Recommend me some movies in the comedy genre", the string output command for that is :
    
        GENRES = ['Comedy']
        
    \n\nTake this for an example. If the text says, "Recommend me some movies in the action and adventure genre", the string output command for that is :
    
        GENRES = ['Action', 'Adventure']
        
    \n\nTake this for an example. If the text says, "I want to watch some nice action flick", (here, you have to understand that this means the user's intent points towards the "Action" genre) the string output command for that is :
    
        GENRES = ['Action']
        
    \n\nTake this for an example. If the text says, "I'm in the mood for a nice rom-com", (here, you have to understand that this means the user's intent points towards the "Romance" and "Comedy" genre) the string output command for that is :
    
        GENRES = ['Comedy','Romance']
        
    \n\nTake this for an example. If the text says, "How about a nice romantic movie?", (here, you have to understand that this means the user's intent points towards the "Romance" and "Comedy" genre) the string output command for that is :
    
        GENRES = ['Romance']
        
    \n\nTake this for an example. If the text says, "show me some science fiction movies", (here, you have to understand that this means the user's intent points towards the "Romance" and "Comedy" genre) the string output command for that is :
    
        GENRES = ['Sci-Fi']
        
    \n\nTake this for an example. If the text says, "Recommend me some movies with Tom Hanks in it", the string output command for that is :
    
        CAST = ['Tom Hanks']
        
    \n\nTake this for an example. If the text says, "Movies having Margot Robbie and Will Smith in it's cast", the string output command for that is :
    
        CAST = ['Margot Robbie', 'Will Smith']
        
    \n\nTake this for an example. If the text says, "Show some movies with Tom Hanks and Leonardo DiCaprio in them", the string output command for that is :
    
        CAST = ['Tom Hanks', 'Leonardo DiCaprio']
        
    \n\nTake this for an example. If the text says, "Recommend me some movies with superhero and aliens type",(you have to understand that the user's intent here is a bit specific than abstract, so it lies more in keywords than genre) the string output command for that is :
    
        KEYWORDS = ['superhero', 'aliens']
        
    \n\nTake this for an example. If the text says, "I am in the mood for seeing kind of movies with a female protaganist",(you have to understand that the user's intent here is a bit specific than abstract, so it lies more in keywords than genre) the string output command for that is :
    
        KEYWORDS = ['female protaganist']
        
    \n\nTake this for an example. If the text says, "show me some movies of angry protaganist kind",(you have to understand that the user's intent here is a bit specific than abstract, so it lies more in keywords than genre) the string output command for that is :
    
        KEYWORDS = ['angry protaganist']
        
    \n\nTake this for an example. If the text says, "How about some movies with battle in keywords?",(you have to understand that the user's intent here is a bit specific than abstract, so it lies more in keywords than genre) the string output command for that is :
    
        KEYWORDS = ['battle']
    
      
    \n\nAlso, make sure that the SQL code / stirng output does not have ``` in the beginning or the end of your answer, and it should'nt even have "" anywhere in the beginning or end of the answer.
    Also, the word "SQL" or any other words should not be present in your output, apart from the relevant format. Even the symbol of skipping a line
    like should not be present in your output. Just the commands or output as you have been shown above in the examples.
    Just the commands or output as you have been shown above in the examples.
    """,
    """
    You are an expert in variable declaration and handling in Python. Here, you have been a variable and their values.
    Whenever you are given a text query, you have to convert the values of a text query as a string.
    Now, this string is suppose to later converted to a list of strings. You have to make sure that the string is in the correct format.
    Below given are a few examples of how you can convert the text query into a string. You will generally be given a text query with 
    variable names from "TITLE","GENRES","CAST", "KEYWORDS" and you have to convert their values into a string of lists.
    
    
    \n\nTake this for an example. If the text says, "TITLE = ['Guardians of the Galaxy Vol. 2']", the string output command for that is :
        ['Guardians of the Galaxy Vol. 2']
    
    \n\nTake this for an example. If the text says, "TITLE = ['Avengers : Infinity War']", the string output command for that is :
        ['Avengers : Infinity War']
    
    \n\nTake this for an example. If the text says, "GENRES = ['Comedy']", the string output command for that is :
        ['Comedy']
        
    \n\nTake this for an example. If the text says, "GENRES = ['Action', 'Adventure']", the string output command for that is :
        ['Action', 'Adventure']
        
    \n\nTake this for an example. If the text says, "GENRES = ['Comedy','Romance']", the string output command for that is :
        ['Comedy','Romance']
        
    \n\nTake this for an example. If the text says, "GENRES = ['Romance']", the string output command for that is :
        ['Romance']
        
    \n\nTake this for an example. If the text says, "GENRES = ['Sci-Fi']", the string output command for that is :
        ['Sci-Fi']
        
    \n\nTake this for an example. If the text says, "GENRES = ['Action']", the string output command for that is :
        ['Action']
        
    \n\nTake this for an example. If the text says, "CAST = ['Tom Hanks']", the string output command for that is :
        ['Tom Hanks']
        
    \n\nTake this for an example. If the text says, "CAST = ['Margot Robbie', 'Will Smith']", the string output command for that is :
        ['Margot Robbie', 'Will Smith']
        
    \n\nTake this for an example. If the text says, "CAST = ['Tom Hanks', 'Leonardo DiCaprio']", the string output command for that is :
        ['Tom Hanks', 'Leonardo DiCaprio']
        
    \n\nTake this for an example. If the text says, "KEYWORDS = ['superhero', 'aliens']", the string output command for that is :
        ['superhero', 'aliens']
        
    \n\nTake this for an example. If the text says, "KEYWORDS = ['female protaganist']", the string output command for that is :
        ['female protaganist']
        
    \n\nTake this for an example. If the text says, "KEYWORDS = ['angry protaganist']", the string output command for that is :
        ['angry protaganist']
        
    \n\nTake this for an example. If the text says, "KEYWORDS = ['battle']", the string output command for that is :
        ['battle']
        
    \n\nRemember, the output should be a string and not a list. The list is just for your understanding. The output should be a string, which will be 
    converted to a list later on using "eval()" in Python, so the output should NOT have  ``` OR python or any other extra symbols in it's output.
    MAKE SURE IT IS A STRING OUTPUT ONLY, AS YOUR RESPONSE WILL BE WORKED ON LATER ON PYTHON FUNCTIONS.
    Also, make sure that the string output does not have ``` in the beginning or the end of your answer. Also, the word "STRING" or any
    other words should not be present in your output. Just the commands or output as you have been shown above in the examples.
    """
]

Now, to set the context for the function to differentiate between the intent of the query, we will set up some knowledge bases for the function to refer to.

In [9]:
query_based_keywords = ['genre', 'genres', 'cast', 'actor', 'actress', 'keyword','in it','in them','type','kind']
title_based_keywords = ["title","similar","like"]

# Query Inferencing using Google Gemini API

In [10]:
def subArray(arr1,arr2):
    # TC : O(arr1.length + arr2.length), SC : O(arr2.length)
    mp = map(str.lower,arr2)
    
    for i in arr1:
        if(i.lower() not in mp):
            return False
        
    return True


def get_gemini_response(question,prompts):
    try:
        model = genai.GenerativeModel('gemini-1.5-flash-latest')
        safe = [
            {
                "category": "HARM_CATEGORY_DANGEROUS",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_HARASSMENT",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_HATE_SPEECH",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                "threshold": "BLOCK_NONE",
            },
        ]
        
        response_level1 = model.generate_content([prompts[0],question],safety_settings = safe)
        
        if("SELECT" in response_level1.text):
            return response_level1.text.replace('\n', '').replace(';', '')
        
        all_movies = database_query_based.values
        
        for query_keyword in query_based_keywords:
            if(query_keyword in question.lower()):
                if(query_keyword == "genres" or query_keyword.lower() == "genre"):
                    response_level2 = model.generate_content([prompts[1],response_level1.text],safety_settings = safe)
                    query_genres = [genre.lower() for genre in eval(response_level2.text)]
                    
                    final_movie_titles = []
                    for movie in all_movies:
                        current_movie_genre = [genre.lower() for genre in eval(movie[4])]
                        if(subArray(query_genres,current_movie_genre)==True):
                            final_movie_titles.append(movie[2])
                        
                    return final_movie_titles[:10]
                
                elif(query_keyword.lower() == "cast" or query_keyword.lower() == "actor" or query_keyword.lower() == "actress" or query_keyword.lower() == "in it" or query_keyword.lower() == "in them"):
                    response_level2 = model.generate_content([prompts[1],response_level1.text],safety_settings = safe)
                    query_cast = [cast.lower() for cast in  eval(response_level2.text)]
                    
                    final_movie_titles = []
                    for movie in all_movies:
                        current_movie_cast = [cast.lower() for cast in eval(movie[6])]
                        if(subArray(query_cast,current_movie_cast)==True):
                            final_movie_titles.append(movie[2])
                        
                    return final_movie_titles[:10]
                
                elif(query_keyword.lower() == "keyword" or query_keyword.lower() == "type" or query_keyword.lower() == "kind"):
                    response_level2 = model.generate_content([prompts[1],response_level1.text],safety_settings = safe)
                    query_keywords = [keyword.lower() for keyword in eval(response_level2.text)]
                    
                    final_movie_titles = []
                    for movie in all_movies:
                        current_movie_keywords = [keyword.lower() for keyword in eval(movie[8])]
                        if(subArray(query_keywords,current_movie_keywords)==True):
                            final_movie_titles.append(movie[2])
                        
                    return final_movie_titles[:10]
                
        
        for query_keyword in title_based_keywords:
            if(query_keyword.lower() in question.lower()):
                response_level2 = model.generate_content([prompts[1],response_level1.text],safety_settings = safe)
                query_title = eval(response_level2.text)[0]
                
                return find_similar_movies(query_title,database_key_based,movie_collection,6)    
            
    except Exception as e:
        print(f"Error occurred during request: {e}")
        return ""
            

# Testing Sample Queries

In [12]:
question = "Recommend me some movies with Tom Hanks in it"
response = get_gemini_response(question,prompts)
process_sql_query(response, database)

Toy Story 4
The Circle
The Post
A Beautiful Day in the Neighborhood
Spielberg


In [13]:
question = "Recommend me some movies in the comedy genre"
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

Coco
Fast & Furious Presents: Hobbs & Shaw
Frozen II
Cars 3
Thor: Ragnarok
Shazam!
Murder Mystery
Deadpool 2
Kingsman: The Golden Circle
Toy Story 4


In [14]:
question = "Recommend me some movies in the comedy and adventure genre"
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

Coco
Deadpool 2
Ralph Breaks the Internet
Despicable Me 3
The Angry Birds Movie 2
The Christmas Chronicles
Missing Link
Animal Crackers
Sharknado 5: Global Swarming
Playmobil: The Movie


In [15]:
question = "Recommend me some movies of superhero type"
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

Avengers: Infinity War
Avengers: Endgame
Spider-Man: Homecoming
Spider-Man: Into the Spider-Verse
Thor: Ragnarok
Spider-Man: Far From Home
Shazam!
Deadpool 2
Aquaman
Justice League


In [16]:
question = "Recommend me some movies having female protagonist as keyword" 
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

Frozen II
Maleficent: Mistress of Evil
Captain Marvel
Beauty and the Beast
Alita: Battle Angel
Terminator: Dark Fate
Ready or Not
Tomb Raider
Booksmart
Midsommar


In [17]:
question = "Give me some movies of superheroine type"
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

Avengers: Infinity War
Spider-Man: Into the Spider-Verse
Wonder Woman
Incredibles 2
Captain Marvel


In [18]:
question = "Show me some movies having voting average above 8"
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

Avengers: Infinity War
Avengers: Endgame
Coco
Spider-Man: Into the Spider-Verse
Joker
Five Feet Apart
Call Me by Your Name
Ford v Ferrari
Green Book
Jojo Rabbit


In [19]:
question = "I wanna see some movies with lowest voting average from the year 2018"
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

The Second Coming of Christ
The Open House
The Con Is On
Holmes & Watson
Occupation
Beyond White Space
Looking Glass
Edge of Fear
Siberia
Air Strike


In [20]:
question = "Show me top 5 movies based on voting average released post 2017"
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

Spider-Man: Into the Spider-Verse
Five Feet Apart
Klaus
Taylor Swift: Reputation Stadium Tour
Avengers: Endgame


In [32]:
question = "Can you recommend me similar movies similar to Avengers: Endgame?"
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

Avengers: Infinity War
Spider-Man: Homecoming
Guardians of the Galaxy Vol. 2
Thor: Ragnarok
Ant-Man and the Wasp


In [33]:
question = "Show me some movies like Captain Marvel"
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

Venom
Black Panther
Guardians of the Galaxy Vol. 2
Spider-Man: Into the Spider-Verse
Justice League


In [34]:
question = "I really liked the movie with the title Alien: Covenant, can you give me something on similar lines?"
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

Life
High Life
Blade Runner 2049
Venom
Valerian and the City of a Thousand Planets


With this, we now have integrated the Google Gemini API to infer the intent of the query and convert it into tangible SQL queries and/or chroma DB queries. These can of course be further processed to get the desired results, as well as prompt improvements are highly encouraged for better results.